In [2]:
import serial #Import python serial library
from datetime import datetime
from influxdb_client import InfluxDBClient, Point, WritePrecision
from influxdb_client.client.write_api import SYNCHRONOUS
# You can generate an API token from the "API Tokens Tab" in the UI
token = "_3UiGvqpb8fS3gsX1eAebnKgoe5-ERvEdbQFVUeXGEksM2D4RVYsFtuDvEvLSNvETjIEZ7ICPhkLLkdjHWi6iA=="
org = "UQTR"
bucket = "Battery_Data"

influxClient= InfluxDBClient(url="http://localhost:8086", token=token, org=org)
write_api = influxClient.write_api(write_options=SYNCHRONOUS)


#serialPort = serial.Serial("COM3",9600,timeout=1,parity=serial.PARITY_NONE, stopbits=serial.STOPBITS_ONE, bytesize=serial.EIGHTBITS) #open Serial Port
serialPort = serial.Serial("COM7",9600,timeout=1,parity=serial.PARITY_NONE, stopbits=serial.STOPBITS_ONE, bytesize=serial.EIGHTBITS) #Ubuntu open Serial Port
i=0
print(datetime.utcnow())
Continue=True;
while Continue:
    try:
        #Read a single ine from serial (Message that ends by LF \n)
        data=serialPort.readline()
        dataString = data.replace(b'\x00', b'').decode('ascii') #Delete null characters
        if data==0x00:
            print("No Data")
        else:
            #Process and Transform data from MCU
            timestamp=datetime.utcnow()
            i=i+1
            dataList=dataString.split(",")
            currentInfo=dataList[0].split("=")
            voltageInfo=dataList[1].split("=")
            if(currentInfo[0]=="Current"):
                currentValue=float(currentInfo[1])
            if(voltageInfo[0]== "Voltage"):
                voltageValue=float(voltageInfo[1])
            #Save Time Series to Influxdb
            point = Point("PowerMeasurement")\
            .field("current", currentValue)\
            .field("voltage",voltageValue)\
            .time(timestamp,write_precision='ns')
            print(datetime.utcnow())
            write_api.write(bucket, org, point)
    except KeyboardInterrupt:
        print("Stop requested by User")
        Continue=False
    except: 
        print("An Error has occurred... Sending Previous Data")
        point = Point("PowerMeasurement")\
        .field("current", currentValue)\
        .field("voltage",voltageValue)\
        .time(timestamp,write_precision='ns')
        print(datetime.utcnow())
        write_api.write(bucket, org, point)
            
            
print(datetime.utcnow())     
serialPort.close()
print("Port Closed")
influxClient.close()
print("client Close")



2022-05-26 20:35:43.628282
2022-05-26 20:35:43.697801
2022-05-26 20:35:43.797193
2022-05-26 20:35:43.897267
2022-05-26 20:35:43.996206
2022-05-26 20:35:44.095613
2022-05-26 20:35:44.197197
2022-05-26 20:35:44.296476
2022-05-26 20:35:44.396749
2022-05-26 20:35:44.496883
2022-05-26 20:35:44.597224
2022-05-26 20:35:44.695674
2022-05-26 20:35:44.796088
2022-05-26 20:35:44.895738
2022-05-26 20:35:44.995125
2022-05-26 20:35:45.095118
2022-05-26 20:35:45.196171
2022-05-26 20:35:45.295736
2022-05-26 20:35:45.397700
2022-05-26 20:35:45.495575
2022-05-26 20:35:45.595766
2022-05-26 20:35:45.695442
2022-05-26 20:35:45.796778
2022-05-26 20:35:45.895812
2022-05-26 20:35:45.996065
2022-05-26 20:35:46.095008
2022-05-26 20:35:46.195986
2022-05-26 20:35:46.295406
2022-05-26 20:35:46.396660
2022-05-26 20:35:46.497538
2022-05-26 20:35:46.595799
2022-05-26 20:35:46.695792
2022-05-26 20:35:46.796567
2022-05-26 20:35:46.895539
2022-05-26 20:35:46.995464
2022-05-26 20:35:47.095511
2022-05-26 20:35:47.195782
2

In [114]:
serialPort.close()
print("Port Closed")

Port Closed
